<a href="https://colab.research.google.com/github/shreySham04/roommate-matcher/blob/main/Roomer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [5]:
# Load the CSV file
df = pd.read_csv('roomy.csv')

# Clean column names
df.columns = [col.strip() for col in df.columns]

# Function to calculate compatibility score between two users
def calculate_compatibility(user1, user2):
    score = 0
    reasons = []

    # City match
    city1 = str(user1['Preferred City\\Area to live in']).lower()
    city2 = str(user2['Preferred City\\Area to live in']).lower()
    if city1 and city2 and city1 == city2:
        score += 5
        reasons.append("Same city preference")

    # Rent budget match
    try:
        rent1 = int(''.join(filter(str.isdigit, str(user1['Monthly Rent Budget']))))
        rent2 = int(''.join(filter(str.isdigit, str(user2['Monthly Rent Budget']))))
        if abs(rent1 - rent2) / max(rent1, rent2) <= 0.2:
            score += 5
            reasons.append("Similar rent budget")
    except:
        pass

    # Sleeping schedule
    if str(user1['Sleeping Schedule']).strip().lower() == str(user2['Sleeping Schedule']).strip().lower():
        score += 3
        reasons.append("Same sleeping schedule")

    # Smoking habits
    if str(user1['Smoking Habits']).strip().lower() == str(user2['Smoking Habits']).strip().lower():
        score += 2
        reasons.append("Same smoking habit")

    # Drinking habits
    if str(user1['Drinking Habits']).strip().lower() == str(user2['Drinking Habits']).strip().lower():
        score += 2
        reasons.append("Same drinking habit")

    # Cleanliness preference
    try:
        clean1 = int(user1['Cleanliness Preference'])
        clean2 = int(user2['Cleanliness Preference'])
        if abs(clean1 - clean2) <= 1:
            score += 3
            reasons.append("Similar cleanliness preference")
    except:
        pass

    # Cooking habits
    if str(user1['Cooking Habits']).strip().lower() == str(user2['Cooking Habits']).strip().lower():
        score += 2
        reasons.append("Same cooking habits")

    # Pet friendliness
    pet1 = str(user1['Pets']).strip().lower()
    pet2 = str(user2['Pets']).strip().lower()
    if ("open" in pet1 and "open" in pet2) or ("not" in pet1 and "not" in pet2):
        score += 2
        reasons.append("Similar pet preference")

    # Hobbies / interests
    hobbies1 = set(str(user1['Hobbies/Interests']).lower().split(","))
    hobbies2 = set(str(user2['Hobbies/Interests']).lower().split(","))
    common = hobbies1 & hobbies2
    if common:
        hobby_score = min(5, len(common)) * 3
        score += hobby_score
        reasons.append(f"Common hobbies: {', '.join(common)}")

    # Preferred roommate gender compatibility
    pref1 = str(user1['Preferred Roommate Gender']).strip().lower()
    gender2 = str(user2['Gender']).strip().lower()
    pref2 = str(user2['Preferred Roommate Gender']).strip().lower()
    gender1 = str(user1['Gender']).strip().lower()

    compatible = False
    if (pref1 == 'any' or pref1 == gender2) and (pref2 == 'any' or pref2 == gender1):
        compatible = True

    if compatible:
        score += 3
        reasons.append("Roommate gender preference compatible")

    return score, reasons

# Function to find matches for a specific user by serial number
def find_matches_for_user(serial_no, top_n=3):
    index = serial_no - 2  # Adjusting for header row and zero-indexing
    if index < 0 or index >= len(df):
        print("Invalid serial number!")
        return

    user = df.iloc[index]
    user_matches = []
    for idx2, other in df.iterrows():
        if index == idx2:
            continue
        score, reasons = calculate_compatibility(user, other)
        user_matches.append((other['Full Name'], score, reasons))

    user_matches.sort(key=lambda x: x[1], reverse=True)

    print(f"\nTop matches for {user['Full Name']} (Serial No: {serial_no}):")
    for match_name, score, reasons in user_matches[:top_n]:
        print(f"\n -> {match_name} (Score: {score})")
        for reason in reasons:
            print(f"   - {reason}")

# Example usage:
serial_no = int(input("Enter the serial number of the user: "))
find_matches_for_user(serial_no)


Enter the serial number of the user: 2

Top matches for G.SAKETH REDDY (Serial No: 2):

 -> THIGULLA CHATRAPATHI REDDY  (Score: 12)
   - Same smoking habit
   - Similar cleanliness preference
   - Same cooking habits
   - Similar pet preference
   - Roommate gender preference compatible

 -> Mohiuddin Ahmed  (Score: 12)
   - Same drinking habit
   - Similar cleanliness preference
   - Same cooking habits
   - Similar pet preference
   - Roommate gender preference compatible

 -> meghanadh (Score: 12)
   - Similar rent budget
   - Same smoking habit
   - Same drinking habit
   - Similar cleanliness preference
